## Try out neighbor

In [38]:
import pandas as pd
import networkx as nx
import numpy as np

import os,sys

sys.path.insert(0,'../utils/')
from graph import n_nearest_neighbors
from model_data_prep import data_prepare_lstm

Using TensorFlow backend.


In [2]:
region_time_series_filtered = pd.read_csv('../../output/region_time_series_filtered.csv', index_col = 0)
region_time_series_filtered.head()

,2016-02-01 00:00:00,2016-02-01 01:00:00,2016-02-01 02:00:00,2016-02-01 03:00:00,2016-02-01 04:00:00,2016-02-01 05:00:00,2016-02-01 06:00:00,2016-02-01 07:00:00,2016-02-01 08:00:00,2016-02-01 09:00:00,...,2016-02-29 14:00:00,2016-02-29 15:00:00,2016-02-29 16:00:00,2016-02-29 17:00:00,2016-02-29 18:00:00,2016-02-29 19:00:00,2016-02-29 20:00:00,2016-02-29 21:00:00,2016-02-29 22:00:00,2016-02-29 23:00:00
region_6075010100,1.103094,0.850827,0.137040,0.078909,0.196874,1.539069,1.714272,2.258792,1.799093,2.151459,...,2.045441,2.239105,2.471772,2.591621,2.202300,1.242359,1.202609,0.981914,1.050426,1.368480
region_6075010200,4.474976,6.177880,7.323182,6.249900,5.763750,4.905341,4.557325,3.481254,1.850865,2.825775,...,2.397086,2.734535,2.477057,2.435120,3.243975,3.458722,2.948414,2.544947,2.356152,2.466459
region_6075010300,5.635000,5.944400,8.889000,1.620333,0.000000,5.532250,4.415793,4.503976,4.671350,5.090225,...,3.904699,4.047194,4.435053,4.571130,4.656116,4.763261,5.180390,5.349852,4.901961,4.934529
region_6075010400,5.674429,9.444333,6.944000,7.500000,12.778000,5.926167,4.881000,5.593897,3.988920,6.063925,...,4.967918,4.736745,4.836363,4.627703,4.603434,4.804670,4.637204,5.731026,4.656882,4.882050
region_6075010500,3.446908,6.550136,4.444000,0.000000,5.799818,5.608138,4.122118,4.304811,4.170099,4.161973,...,3.743409,4.107213,3.237954,2.903063,3.350814,3.964277,4.254140,4.476809,4.541343,4.188110


In [3]:
region_time_series_filtered.mean().to_frame().T

,2016-02-01 00:00:00,2016-02-01 01:00:00,2016-02-01 02:00:00,2016-02-01 03:00:00,2016-02-01 04:00:00,2016-02-01 05:00:00,2016-02-01 06:00:00,2016-02-01 07:00:00,2016-02-01 08:00:00,2016-02-01 09:00:00,...,2016-02-29 14:00:00,2016-02-29 15:00:00,2016-02-29 16:00:00,2016-02-29 17:00:00,2016-02-29 18:00:00,2016-02-29 19:00:00,2016-02-29 20:00:00,2016-02-29 21:00:00,2016-02-29 22:00:00,2016-02-29 23:00:00
0,6.890707,7.347529,5.722829,5.124776,6.597469,7.147443,6.802715,5.955805,5.605728,5.848328,...,5.461527,5.456205,5.486594,5.463851,5.656708,5.958119,5.994071,6.157814,6.305097,6.4387


In [37]:
def average_nb_ts(sample_time_series):
    # use graph to retrieve the neighbor of one target region, and get the average time series for all regions
    G = nx.read_gpickle('../../output/census_filtered.gpickle')
    geoids = [i.strip('region_') for i in list(sample_time_series.index)]
    nbs_df = pd.DataFrame()

    for geoid in geoids:
        nb_list = n_nearest_neighbors(G, geoid)
        neighbor_df = pd.DataFrame()

        for x in nb_list:
            neighbor_data = sample_time_series.loc['region_' + str(x)].to_frame().T
            neighbor_df = pd.concat([neighbor_df, neighbor_data])
        #average for one region neighbors
        nb_mean = neighbor_df.mean().to_frame().T
        # concat all neighbor information for different regions together.
        nbs_df = pd.concat([nbs_df,nb_mean])
    nbs_df = nbs_df.set_index(sample_time_series.index)
    return nbs_df
average_nb_ts(region_time_series_filtered)

,2016-02-01 00:00:00,2016-02-01 01:00:00,2016-02-01 02:00:00,2016-02-01 03:00:00,2016-02-01 04:00:00,2016-02-01 05:00:00,2016-02-01 06:00:00,2016-02-01 07:00:00,2016-02-01 08:00:00,2016-02-01 09:00:00,...,2016-02-29 14:00:00,2016-02-29 15:00:00,2016-02-29 16:00:00,2016-02-29 17:00:00,2016-02-29 18:00:00,2016-02-29 19:00:00,2016-02-29 20:00:00,2016-02-29 21:00:00,2016-02-29 22:00:00,2016-02-29 23:00:00
region_6075010100,6.480768,8.823370,8.059260,5.772889,7.132197,7.055479,6.089069,6.252809,5.520378,6.308938,...,6.127099,5.793443,5.685669,5.183558,5.542998,6.272635,6.205025,6.592752,5.975072,6.372854
region_6075010200,6.248849,5.888219,6.418947,5.241159,6.398005,5.638734,5.332781,5.303856,5.093766,5.193438,...,5.053989,4.973830,4.981970,4.937349,4.809727,5.174688,5.277762,5.619552,5.576275,5.910977
region_6075010300,3.477993,5.329448,5.124476,3.750632,6.074004,3.915525,3.312451,3.582053,2.917374,3.630636,...,3.024607,3.238095,3.289057,3.309855,3.191434,3.097492,3.080672,3.182356,3.093266,3.225322
region_6075010400,3.859341,5.480273,5.249508,2.450948,4.199338,4.546985,3.614004,3.815284,3.319742,3.951376,...,3.245631,3.339117,3.389749,3.328969,3.266088,3.396526,3.638673,3.885822,3.839029,3.762576
region_6075010500,5.719947,7.592601,6.034659,5.853844,6.202205,5.560932,5.124495,5.200162,4.548310,5.220212,...,5.365853,4.799898,4.919642,4.494143,4.496650,5.063265,5.141349,5.797532,5.474762,5.731782
region_6075010600,4.647360,7.825055,5.381750,5.925917,6.519455,5.225496,4.314999,4.398190,3.749549,4.462382,...,3.872302,3.823783,3.713775,3.613019,3.683933,3.980209,4.126989,4.745849,4.659739,4.496789
region_6075010700,4.038812,4.838412,4.661463,3.174717,3.535067,3.671491,3.305163,3.563510,3.267109,4.008644,...,3.417020,3.231466,3.371839,3.408318,3.325473,3.534379,3.669852,4.052314,3.687939,3.774918
region_6075010800,3.934166,5.103833,5.957885,2.724207,4.547200,4.676821,3.751763,4.023573,4.091018,4.086231,...,3.441461,3.669397,3.888438,3.861671,3.726968,3.768343,3.980519,3.990594,3.851122,4.152171
region_6075010900,4.833098,4.924700,4.875521,3.504459,4.982491,4.318907,3.984592,3.600232,3.450909,3.673861,...,3.135721,3.365644,3.384798,3.606416,3.712662,3.707046,3.842142,3.983825,3.749104,3.975676
region_6075011000,3.519409,4.485331,2.912010,1.503563,3.818813,4.500147,3.663494,3.606469,3.862091,3.794375,...,3.197262,3.482357,3.607522,3.684760,3.414311,3.294387,3.235805,3.125234,3.017157,3.646475


In [41]:
x_train, y_train, X_test, Y_test = data_prepare_lstm(region_time_series_filtered, split_size=0.7, time_window=12)
x_train.shape

(92625, 12, 1)

In [50]:
import random
a = np.random.normal(size = (1000,2,1))
b = np.random.normal(size = (1000,2,1))
np.concatenate((a, b), axis=1).shape

(1000, 4, 1)

## Keras model trouble shooting

In [54]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [65]:
nbs_df= pd.read_csv('../../output/neighbor_ts.csv',index_col=0)
split_size = 0.7
sliding_window = 12
epoch=8
# generate target time series train data
x_train, y_train, X_test, Y_test = data_prepare_lstm(region_time_series_filtered, split_size=split_size, time_window=sliding_window)
x_train_nb, _, X_test_nb, _ = data_prepare_lstm(nbs_df, split_size=split_size, time_window=sliding_window)

x_train_agg = np.concatenate((x_train, x_train_nb), axis=1)
y_train_agg = y_train
X_test_agg = np.concatenate((X_test, X_test_nb), axis=1)
Y_test_agg = Y_test



model = Sequential()
model.add(LSTM(units=16, return_sequences=True, input_shape=(x_train_agg.shape[1], 1)))
model.add(LSTM(units=16))
model.add(Dense(1))
#model.add(Dense(1,activation='relu'))
print(model.summary())
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
history = model.fit(x_train_agg, y_train_agg, epochs=epoch, validation_split=0.25, batch_size=16, verbose=2)


In [69]:
type(nbs_df.loc['region_6075010100'])

pandas.core.series.Series

In [3]:
i=0
for row in range(40):
    if i<20:
        i+=1
        print(i)
        continue
    i+=1
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


## retrieve train score from log

In [37]:
next_symbel = False
train_mae = []
test_mae = []
with open('../../logs/lstm_1model_1region.log') as f: #lstm_multi_nb_combine.txt lstm_1model_1region.log
    for line in f:
        if next_symbel:
            score = line.split('mean_absolute_error:')[1].split(' ')[1]

            train_mae.append(round(float(score),4))
            next_symbel=False
        if 'Epoch 8/8' in line:

            next_symbel = True
            continue
        if 'Test MAE:' in line:
            score = line.strip('Test MAE: \n')
            test_mae.append(round(float(score),4))
test_mae   

[0.484,
 1.079,
 0.763,
 1.171,
 0.782,
 0.928,
 0.732,
 0.259,
 1.016,
 0.996,
 0.942,
 0.874,
 0.508,
 0.891,
 0.803,
 1.805,
 1.346,
 0.914,
 0.782,
 0.618,
 0.472,
 0.347,
 0.597,
 1.118,
 0.914,
 1.037,
 1.269,
 1.025,
 0.697,
 1.866,
 0.761,
 0.78,
 0.596,
 0.474,
 0.98,
 0.939,
 0.602,
 0.806,
 1.212,
 0.356,
 0.63,
 0.868,
 0.908,
 0.558,
 0.673,
 0.754,
 0.827,
 0.826,
 0.433,
 0.701,
 0.651,
 0.65,
 0.938,
 0.541,
 0.77,
 1.249,
 1.008,
 0.643,
 0.728,
 1.087,
 0.953,
 0.565,
 1.307,
 0.829,
 0.749,
 0.759,
 1.178,
 0.979,
 5.658,
 0.83,
 0.651,
 1.386,
 1.153,
 4.428,
 2.561,
 0.947,
 0.655,
 1.309,
 0.527,
 0.602,
 1.829,
 0.484,
 0.727,
 0.764,
 0.418,
 0.609,
 0.646,
 2.575,
 0.794,
 1.016,
 2.471,
 3.22,
 0.157,
 1.509,
 1.234,
 1.089,
 1.533,
 1.837,
 4.654,
 0.804,
 2.003,
 2.031,
 0.928,
 1.229,
 1.173,
 3.132,
 0.546,
 0.513,
 0.409,
 0.711,
 2.373,
 2.334,
 2.54,
 7.252,
 1.823,
 1.486,
 1.955,
 0.606,
 0.838,
 1.622,
 1.013,
 0.525,
 2.188,
 0.893,
 1.755,
 0.758,


In [35]:
next_symbel = False
train_mae = []
test_mae = []
with open('../../logs/lstm_multi_nb_combine.txt') as f: #lstm_multi_nb_combine.txt lstm_1model_1region.log
    for line in f:
        if next_symbel:
            score = line.split('mean_absolute_error:')[1].split(' ')[1]

            train_mae.append(round(float(score),4))
            next_symbel=False
        if 'Epoch 8/8' in line:

            next_symbel = True
            continue
        if 'Test MAE:' in line:
            score = line.strip('Test MAE: \n')
            test_mae.append(round(float(score),4))
test_mae    

[0.495,
 1.083,
 0.756,
 1.17,
 0.773,
 0.947,
 0.725,
 0.44,
 1.047,
 1.005,
 0.962,
 0.936,
 0.522,
 0.91,
 0.81,
 1.833,
 1.349,
 0.924,
 0.811,
 0.625,
 0.471,
 0.359,
 0.582,
 1.152,
 0.915,
 1.032,
 1.303,
 1.02,
 0.725,
 1.884,
 0.762,
 0.777,
 0.595,
 0.47,
 0.986,
 0.951,
 0.608,
 0.803,
 1.267,
 0.364,
 0.617,
 0.871,
 0.923,
 0.445,
 0.671,
 0.736,
 0.83,
 0.648,
 0.429,
 0.706,
 0.647,
 0.652,
 0.969,
 0.541,
 0.776,
 1.264,
 1.037,
 0.664,
 0.763,
 1.092,
 0.958,
 0.561,
 1.357,
 0.83,
 0.745,
 0.584,
 1.223,
 0.978,
 6.63,
 0.85,
 0.649,
 1.152,
 1.153,
 3.342,
 2.611,
 0.951,
 0.648,
 1.31,
 0.524,
 0.619,
 1.844,
 0.505,
 0.733,
 0.76,
 0.413,
 0.603,
 0.638,
 1.514,
 0.796,
 1.035,
 1.948,
 2.636,
 0.177,
 2.277,
 1.396,
 1.102,
 1.535,
 1.813,
 3.216,
 0.799,
 1.474,
 2.088,
 0.804,
 1.26,
 1.394,
 3.834,
 0.545,
 0.513,
 0.433,
 0.886,
 1.969,
 1.876,
 2.216,
 7.003,
 1.814,
 1.547,
 2.101,
 0.614,
 0.848,
 1.63,
 0.849,
 0.586,
 2.035,
 0.924,
 1.648,
 0.757,
 0.595

In [15]:
import pandas as pd
ts = pd.read_csv('../../output/region_time_series_filtered.csv', index_col = 0)
region_id_list = ts.index.tolist()

In [33]:
import pickle
import numpy as np
def load_pickle(f_name,path=False):
    print("in func load_pickle")
    print(path)
    variable=pickle.load(open(r"%s%s"%(path,f_name), "rb"))
    print('successfully load %s'%(f_name))
    return variable

criteria_scores = [train_mae, test_mae]
weights = load_pickle('region_weights.p',path = '../../output/')
# get ordered subset of weight value
weight_list = [weights[k] for k in region_id_list if k in weights]
weighted_scores = []
for score_list in criteria_scores:
    weighted_scores.append(np.average(score_list, weights=weight_list))
weighted_scores

in func load_pickle
../../output/
successfully load region_weights.p


[1.1312847106544612, 1.0342306031276194]

In [36]:
import pickle
import numpy as np
def load_pickle(f_name,path=False):
    print("in func load_pickle")
    print(path)
    variable=pickle.load(open(r"%s%s"%(path,f_name), "rb"))
    print('successfully load %s'%(f_name))
    return variable

criteria_scores = [train_mae, test_mae]
weights = load_pickle('region_weights.p',path = '../../output/')
# get ordered subset of weight value
weight_list = [weights[k] for k in region_id_list if k in weights]
weighted_scores = []
for score_list in criteria_scores:
    weighted_scores.append(np.average(score_list, weights=weight_list))
weighted_scores

in func load_pickle
../../output/
successfully load region_weights.p


[1.1324872098756738, 1.035456642728073]

In [38]:
split_size = 0.7
sliding_window = 12
file_name = 'region_time_series_filtered.csv'

new_time_df_new = pd.read_csv('../../output/'+file_name, index_col = 0)
print(new_time_df_new.shape)
length_col = len(new_time_df_new)
print(length_col)

# # generate neighbor time series data
# nbs_df = average_nb_ts(new_time_df_new)
# print('finish generate neighbor df')



(195, 696)
195


In [45]:
twitter_df= pd.read_csv('../../output/tweet_features_by_hour.csv',index_col=0).loc['tweet_count'].to_frame().T
print(twitter_df.shape)


(1, 696)


In [46]:
twitter_df_copy =pd.concat([twitter_df]*length_col,ignore_index=True)
print(twitter_df_copy.shape)


(195, 696)


In [47]:
# generate target time series train data
x_train, y_train, X_test, Y_test = data_prepare_lstm(new_time_df_new, split_size=split_size, time_window=sliding_window)
x_train_nb, _, X_test_nb, _ = data_prepare_lstm(twitter_df, split_size=split_size, time_window=sliding_window)

x_train_agg = np.concatenate((x_train, x_train_nb), axis=1)
y_train_agg = y_train
X_test_agg = np.concatenate((X_test, X_test_nb), axis=1)
Y_test_agg = Y_test

print(x_train_agg.shape)
history, lstm_rmse,lstm_mae = LSTM_base(x_train_agg, y_train_agg, X_test_agg, Y_test_agg, epoch=8)
plot_lstm(history,attr='_twitter')

NameError: name 'data_prepare_lstm' is not defined